In [1]:
import pandas as pd
import numpy as np
from faker import Faker
from collections import defaultdict
from sqlalchemy import create_engine
import datetime
import random
import re
from neo4j import GraphDatabase

In [2]:
from numpy import integer


fake=Faker()
Faker.seed(50)

fake_account_info = defaultdict(list)

row_number=50

In [3]:
for i in range(row_number):
    fake_account_info['account_holder_first_name'].append(fake.first_name())
    fake_account_info['account_holder_last_name'].append(fake.last_name())
    fake_account_info['unique_id'].append(fake_account_info['account_holder_first_name'][i]+fake_account_info['account_holder_last_name'][i])
    fake_account_info['address_street'].append(fake.street_address())
    fake_account_info['address_city'].append(fake.city())
    fake_account_info['address_state'].append(fake.state())
    fake_account_info['address_zipcode'].append(fake.postalcode_in_state())
    fake_account_info['account_holder_phone_number'].append(fake.phone_number())
    fake_account_info['SSN'].append(fake.ssn())


In [4]:
df=pd.DataFrame(fake_account_info)
df

,account_holder_first_name,account_holder_last_name,unique_id,address_street,address_city,address_state,address_zipcode,account_holder_phone_number,SSN
0,Tyler,Adams,TylerAdams,751 Walker Point,East Michaelville,Minnesota,21267,159.961.1956,006-30-5529
1,Joseph,Conrad,JosephConrad,74972 Dean Place,South Faithville,South Dakota,83114,688.279.8415x8205,010-76-6734
2,Thomas,Gibbs,ThomasGibbs,91246 Matthew Roads,Port Christianbury,Utah,67084,(796)671-6150x01790,315-41-5682
3,Amber,Sutton,AmberSutton,42631 Alex Curve Apt. 204,Alfredhaven,Arkansas,52373,001-490-276-6582x308,620-37-5495
4,Rebecca,Nguyen,RebeccaNguyen,742 Leslie Light,Victoriaside,Delaware,06287,014-774-4934,852-29-9397
5,Dominique,Gonzalez,DominiqueGonzalez,08723 Heather Stream Suite 048,Port Ericchester,Iowa,87801,001-808-471-2258x30337,296-91-7677
6,Cynthia,Pugh,CynthiaPugh,05031 Dustin Squares,Martinezfort,Indiana,30740,(901)156-9006x7956,799-16-8612
7,Timothy,Sanchez,TimothySanchez,132 Thomas Valley,Alyssashire,Missouri,24292,+1-682-920-8366x455,347-40-5653
8,Kyle,Wells,KyleWells,18858 Russell Plains,South Jasonshire,Wisconsin,82626,505.323.8505x318,411-94-7014
9,Matthew,Kline,MatthewKline,4732 Oneal Vista Suite 754,Rebeccachester,Tennessee,72440,310-859-7058x72467,858-34-3266


In [5]:
account_info=df.values.tolist()

In [12]:
uri = "neo4j://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "test"))

#Deleting existing values and relationships
def clear_db(tx):
    tx.run("MATCH(n) Detach Delete n")
#Creating account credentials node
def create_account_credentials(tx,account_holder_first_name,account_holder_last_name,unique_id,address_street,address_city,address_state,address_zipcode,account_holder_phone_number,ssn):
    tx.run('CREATE(ah:AccountHolder {'
            'UniqueId: $unique_id,'
       		    'FirstName: $account_holder_first_name,'
       		    'LastName: $account_holder_last_name}) '

            'Create(addr:Address{'
                'Street: $address_street,'
                'City: $address_city,'
                'State: $address_state,'
                'ZipCode: $address_zipcode}) '

            'Create(pn:PhoneNumber{PhoneNumber:$account_holder_phone_number}) '

            'Create(ssn:SSN{SSN: $ssn})',
            unique_id=unique_id,account_holder_first_name=account_holder_first_name,account_holder_last_name=account_holder_last_name,account_holder_phone_number=account_holder_phone_number,ssn=ssn,address_street=address_street,address_city=address_city,address_state=address_state,address_zipcode=address_zipcode)

def create_address_and_relationship(tx,unique_id,adress_street,address_city,address_state,address_zipcode):
    tx.run(
        'Match(ah:AccountHolder) '
        'Where ah.UniqueId = $ unique_id '
        'Create(addr:Address{'
        'Street: $adress_street,'
        'City: $address_city,'
        'State: $address_state,'
        'ZipCode: $address_zipcode})'
        'Merge(ah)-[:HAS_ADRESS]->(addr)',
        unique_id=unique_id,adress_street=adress_street,address_city=address_city,address_state=address_state,address_zipcode=address_zipcode)


def create_transaction(tx,account_holder_id,account_holder_id_2,transaction_id,time,amount):
    tx.run(
        'Match(b:Bank{account_holder_id:$account_holder_id})'
        'Match(b1:Bank{account_holder_id:$account_holder_id_2})'
        'Create(b)-[:SEND]->(mt:MoneyTransfer{name:"MoneyTransfer",transaction_id:$transaction_id,time:$time,amount:$amount})-[:SEND]->(b1)',
        account_holder_id=account_holder_id,account_holder_id_2=account_holder_id_2,transaction_id=transaction_id,time=time,amount=amount)

    

with driver.session() as session:

    session.execute_write(clear_db)
    for i in range(len(df)):
        #Creating Account Credentials
        session.write_transaction(create_account_credentials, account_info[i][0],account_info[i][1],account_info[i][2],account_info[i][3],account_info[i][4],account_info[i][5],account_info[i][6],account_info[i][7],account_info[i][8])


    #for i in range(len(account_info)):
    #    session.write_transaction(create_account, account_info[i][0],account_info[i][1],account_info[i][2],account_info[i][3],account_info[i][4],account_info[i][5])

    #for i in range(len(transaction_info)):
    #    session.write_transaction(create_transaction,transaction_info[i][0],transaction_info[i][1],str(i),str(transaction_info[i][3]),transaction_info[i][2],)

driver.close()

/tmp/ipykernel_35218/2466309889.py:52: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_account_credentials, account_info[i][0],account_info[i][1],account_info[i][2],account_info[i][3],account_info[i][4],account_info[i][5],account_info[i][6],account_info[i][7],account_info[i][8])


In [ ]:
        'Match(a:AccountHolder)'
        'Where a.UniqueId == $ unique_id'


CREATE (accountHolder1:AccountHolder {
       		FirstName: "John",
       		LastName: "Doe",
       		UniqueId: "JohnDoe" })

CREATE (accountHolder2:AccountHolder {
			FirstName: "Jane",
			LastName: "Appleseed",
			UniqueId: "JaneAppleseed" })

CREATE (accountHolder3:AccountHolder {
			FirstName: "Matt",
			LastName: "Smith",
			UniqueId: "MattSmith" })

// Create Address
CREATE (address1:Address {
			Street: "123 NW 1st Street",
			City: "San Francisco",
			State: "California",
			ZipCode: "94101" })

// Connect 3 account holders to 1 address
CREATE (accountHolder1)-[:HAS_ADDRESS]->(address1),
       (accountHolder2)-[:HAS_ADDRESS]->(address1),
       (accountHolder3)-[:HAS_ADDRESS]->(address1)

// Create Phone Number
CREATE (phoneNumber1:PhoneNumber { PhoneNumber: "555-555-5555" })

// Connect 2 account holders to 1 phone number
CREATE (accountHolder1)-[:HAS_PHONENUMBER]->(phoneNumber1),
       (accountHolder2)-[:HAS_PHONENUMBER]->(phoneNumber1)

// Create SSN
CREATE (ssn1:SSN { SSN: "241-23-1234" })

// Connect 2 account holders to 1 SSN
CREATE (accountHolder2)-[:HAS_SSN]->(ssn1),
       (accountHolder3)-[:HAS_SSN]->(ssn1)

// Create SSN and connect 1 account holder
CREATE (ssn2:SSN { SSN: "241-23-4567" })<-[:HAS_SSN]-(accountHolder1)

// Create Credit Card and connect 1 account holder
CREATE (creditCard1:CreditCard {
			AccountNumber: "1234567890123456",
			Limit: 5000, Balance: 1442.23,
			ExpirationDate: "01-20",
			SecurityCode: "123" })<-[:HAS_CREDITCARD]-(accountHolder1)

// Create Bank Account and connect 1 account holder
CREATE (bankAccount1:BankAccount {
			AccountNumber: "2345678901234567",
			Balance: 7054.43 })<-[:HAS_BANKACCOUNT]-(accountHolder1)

// Create Credit Card and connect 1 account holder
CREATE (creditCard2:CreditCard {
			AccountNumber: "1234567890123456",
			Limit: 4000, Balance: 2345.56,
			ExpirationDate: "02-20",
			SecurityCode: "456" })<-[:HAS_CREDITCARD]-(accountHolder2)

// Create Bank Account and connect 1 account holder
CREATE (bankAccount2:BankAccount {
			AccountNumber: "3456789012345678",
			Balance: 4231.12 })<-[:HAS_BANKACCOUNT]-(accountHolder2)

// Create Unsecured Loan and connect 1 account holder
CREATE (unsecuredLoan2:UnsecuredLoan {
			AccountNumber: "4567890123456789-0",
			Balance: 9045.53,
			APR: .0541,
			LoanAmount: 12000.00 })<-[:HAS_UNSECUREDLOAN]-(accountHolder2)

// Create Bank Account and connect 1 account holder
CREATE (bankAccount3:BankAccount {
			AccountNumber: "4567890123456789",
			Balance: 12345.45 })<-[:HAS_BANKACCOUNT]-(accountHolder3)

// Create Unsecured Loan and connect 1 account holder
CREATE (unsecuredLoan3:UnsecuredLoan {
			AccountNumber: "5678901234567890-0",
			Balance: 16341.95, APR: .0341,
			LoanAmount: 22000.00 })<-[:HAS_UNSECUREDLOAN]-(accountHolder3)

// Create Phone Number and connect 1 account holder
CREATE (phoneNumber2:PhoneNumber {
			PhoneNumber: "555-555-1234" })<-[:HAS_PHONENUMBER]-(accountHolder3)